# UCI Adult Dataset or Census Income

This is a very popular ML task, with tabular data. The objective is to predict whether income exceeds $50K/yr based on census data. 
Also known as "Census Income" dataset.

The data is old and biased on different ways ... but it can be used opaquely for ML experimentation.



## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

GoMLX provides a simple `adult` library to facilitate downdoaling and preprocessing the data. Data is available in [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).

After downloading the data and validating the checksum (both training and testing), it generates the quantiles for the continuous features, and the vocabularies for the categorical features. It saves all this info for faster restart later in a binary file. So this won't be necessary a second time.

The quantiles are used to calibrate the values, using a piece-wise-lienar calibration, very good for these things. See `layers.PieceWiseLinearCalibration` documentation.

We create a flag `--data` to define the directory where to save the intermediary files: downloaded and preprocessed datasets.
In this examle we set it to `/tmp/gonb/adult`. Verbosity can be contolled with the `--verbosity` flag. 

We set default in Go for these flags, but they can easily be reset for a new run by providing them after the `%%` Jupyter kernel meta-command -- in indicates that the subsequent lines should be put in to a `func main`.


In [2]:
import (
    "flag"
    
    "github.com/gomlx/gomlx/examples/adult"
)

var (
	flagDataDir        = flag.String("data", "~/tmp/uci-adult", "Directory to save and load downloaded and generated dataset files.")
    flagVerbosity = flag.Int("verbosity", 0, "Level of verbosity, the higher the more verbose.")
    flagForceDownload  = flag.Bool("force_download", false, "Force re-download of Adult dataset files.")
    flagNumQuantiles    = flag.Int("quantiles", 100, "Max number of quantiles to use for numeric features, used during piece-wise linear calibration. It will only use unique values, so if there are fewer variability, fewer quantiles are used.")
)

%% --verbosity=2
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)


Sample Categorical: (24.08% positive ratio, 23.86% weighted positive ratio)
	Row 0:	[7 10 5 1 2 5 2 39]
	Row 1:	[6 10 3 4 1 5 2 39]
	Row 2:	[4 12 1 6 2 5 2 39]
	...
	Row 32558:	[4 12 7 1 5 5 1 39]
	Row 32559:	[4 12 5 1 4 5 2 39]
	Row 32560:	[5 12 3 4 6 5 1 39]

Sample Continuous:
	Row 0:	[39 13 2174 0 40]
	Row 1:	[50 13 0 0 13]
	Row 2:	[38 9 0 0 40]
	...
	Row 32558:	[58 9 0 0 40]
	Row 32559:	[22 9 0 0 20]
	Row 32560:	[52 9 15024 0 40]


In [3]:
!ls -lh ~/tmp/uci-adult

total 7.0M
-rw-r--r-- 1 janpf janpf 3.8M Mar 21 09:14 adult.data
-rw-r--r-- 1 janpf janpf 1.3M Mar 21 09:14 adult_data-100_quantiles.bin
-rw-r--r-- 1 janpf janpf 2.0M Mar 21 09:14 adult.test


### Creating Datasets

First we create the GoMLX's `Manager`: it's the object that manages the underlying XLA
setup, connection and execution. It's needed to create tensors.

With that we create the samplers of data that we will use to train and evaluate. They implement 
GoMLX's `train.Dataset` interface, which is what is used by our training loop to draw batches to
train, or our eval loop to draw batches to evaluate.

The inputs are 3 tensors: *categorical values*, *continuous values* and *weights*.

In the cell below we define the `Manager` flags, `BuildSamplers` and printout some samples.

In [4]:
import (
    "flag"
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagNumThreads     = flag.Int("num_threads", -1, "Number of threads for XLA. Leave as -1 to use as many as there are cores.")
    flagNumReplicas    = flag.Int("num_replicas", 1, "Number of replicas for XLA. Leave as 1 for now.")
    flagPlatform       = flag.String("platform", "", "Platform to use, if empty uses the default one. Usually 'Host' "+
                                     "(for CPU), 'CUDA' (for CPU) or 'TPU'")
    flagBatchSize      = flag.Int("batch", 128, "BatchSampler size for training")
)

// BuildDatasets returns 3 `train.Dataset`:
// * trainingSampler is an endless random sampler used for training.
// * trainingEvalSampler samples through exactly one epoch of the train dataset.
// * testEvalSampler samples through exactly one epoch of the test dataset.
func BuildDatasets(manager *Manager) (trainingDS, trainingEvalDS, testEvalDS train.Dataset) {
    trainingDS = adult.NewDataset("batched train", adult.Data.Train, manager, *flagBatchSize)
    trainingEvalDS = adult.NewDataset("train", adult.Data.Train, manager, -1)
    testEvalDS = adult.NewDataset("test", adult.Data.Test, manager, -1)
    return
}

// AssertNoError checks that err is nil, otherwise if `log.Fatal` with the error message.
func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

// PositiveRatio finds out the the ratio of positive labels in the
// training and testing data.
//
// We could do this easily with GoMLX computation model (just `ReduceAllSum`), but
// this examples shows it's also ok to mix Go computations.
func PositiveRatio(ds train.Dataset) float32 {
    ds.Reset()  // Start from beginning.
    var sum float32
    var count float32
    for {
        _, _, labels, err := ds.Yield()
        if err == io.EOF {
            break;
        }
        AssertNoError(err)
        data := tensor.Data[float32](labels[0].Local())
        for _, value := range data {
            sum += value
        }
        count += float32(len(data))
    }
    return sum/count
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
trainingDS, trainingEvalDS, testEvalDS := BuildDatasets(manager)

// Take one batch.
_, inputs, labels, err := trainingDS.Yield()
AssertNoError(err)
fmt.Printf("Inputs of batch (size %d):\n", *flagBatchSize)
fmt.Printf("\tcategorical:\n\t\tFeatures=%v\n", adult.Data.VocabulariesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[0].Local().StringN(16))
fmt.Printf("\tcontinuous:\n\t\tFeatures=%v\n", adult.Data.QuantilesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[1].Local().StringN(10))
fmt.Printf("\tweights: %s\n", inputs[2].Local().StringN(5))
fmt.Printf("\nLabels of batch:\n\t%s\n", labels[0].Local().StringN(10))

fmt.Printf("\nLabels distributions:\n\tTrain:\t%.2f%% positive\n\tTest:\t%.2f%% positive\n",
           PositiveRatio(trainingEvalDS)*100.0, PositiveRatio(testEvalDS)*100.0)


Inputs of batch (size 128):
	categorical:
		Features=[workclass education marital-status occupation relationship race sex native-country]
		Values: (Int64)[128 8]: (... too large, 1024 values ..., first 16 values: [6 10 1 12 2 5 1 39 4 1 5 3 3 5 2 39])
	continuous:
		Features=[age education-num capital-gain capital-loss hours-per-week]
		Values: (Float32)[128 5]: (... too large, 640 values ..., first 10 values: [47 13 0 0 50 22 6 0 0 30])
	weights: (Float32)[128 1]: (... too large, 128 values ..., first 5 values: [106252 106843 191204 340880 170086])

Labels of batch:
	(Float32)[128 1]: (... too large, 128 values ..., first 10 values: [0 0 0 0 1 1 0 0 1 0])

Labels distributions:
	Train:	24.08% positive
	Test:	23.62% positive


## Model Definition

Lots of hyper-parameter flags, but otherwise a straight forward FNN, using piece-wise linear calibration of the continuous features, and embeddings for the categorical features.

> **Note**: building models is a constant checking that shapes are compatible. It's a bit annoying, in particular because shapes are known in runtime only -- no compile time check. GoMLX tries to help providing a stack trace of where errors happen so one can pin-point issues quickly. But often it involves lots of experimentation (more than ordinary Go code).
>
> Developing with a Noteboook (see [GoNB](https://github.com/janpfeifer/gonb)) or simply a unit test on your `ModelGraph` function are quick/convenient ways to develop models -- before actually training them.

In [5]:
import (
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/ml/train/optimizers"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    // ModelDType used for the model. Must match RawData Go types.
    ModelDType = shapes.Float32
    
    // Model hyperparameters.
    flagUseCategorical       = flag.Bool("use_categorical", true, "Use categorical features.")
    flagUseContinuous        = flag.Bool("use_continuous", true, "Use continuous features.")
    flagTrainableCalibration = flag.Bool("trainable_calibration", true, "Allow piece-wise linear calibration to adjust outputs.")
    flagEmbeddingDim    = flag.Int("embedding_dim", 8, "Default embedding dimension for categorical values.")
    flagNumHiddenLayers = flag.Int("hidden_layers", 8, "Number of hidden layers, stacked with residual connection.")
    flagNumNodes        = flag.Int("num_nodes", 32, "Number of nodes in hidden layers.")
    flagDropoutRate     = flag.Float64("dropout", 0, "Dropout rate")
    
    // Training parameter, referenced here.
    flagLearningRate    = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagNumSteps       = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
)


// ModelGraph outputs the logits (not the probabilities). The parameter inputs should contain 3 tensors:
//
// - categorical inputs, shaped  `(int64)[batch_size, len(VocabulariesFeatures)]`
// - continuous inputs, shaped `(float32)[batch_size, len(Quantiles)]`
// - weights: not currently used, but shaped `(float32)[batch_size, 1]`.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used, since the dataset is always the same.
    g := inputs[0].Graph()
    
    // Use Cosine schedule of the learning rate.
    optimizers.CosineAnnealingSchedule(ctx, g, ModelDType).
        PeriodInSteps(*flagNumSteps/3).Done()
    
    categorical, continuous := inputs[0], inputs[1]
    batchSize := categorical.Shape().Dimensions[0]
    
    var allEmbeddings []*Node
    graph := categorical.Graph()

    if *flagUseCategorical {
        // Embedding of categorical values, each with its own vocabulary.
        numCategorical := categorical.Shape().Dimensions[1]
        for catIdx := 0; catIdx < numCategorical; catIdx++ {
            // Take one column at a time of the categorical values.
            split := Slice(categorical, AxisRange(), AxisRange(catIdx, catIdx+1))
            // Embed it accordingly.
            embedCtx := ctx.In(fmt.Sprintf("categorical_%d_%s", catIdx, adult.Data.VocabulariesFeatures[catIdx]))
            vocab := adult.Data.Vocabularies[catIdx]
            vocabSize := len(vocab)
            embedding := layers.Embedding(embedCtx, split, ModelDType, vocabSize, *flagEmbeddingDim)
            if !embedding.AssertDims(batchSize, *flagEmbeddingDim) {  // 2-dim tensor, with batch size as the leading dimension.
                return nil
            }
            allEmbeddings = append(allEmbeddings, embedding)
        }
    }

    if *flagUseContinuous {
        // Piecewise-linear calibration of the continuous values. Each feature has its own number of quantiles.
        numContinuous := continuous.Shape().Dimensions[1]
        for contIdx := 0; contIdx < numContinuous; contIdx++ {
            // Take one column at a time of the continuous values.
            split := Slice(continuous, AxisRange(), AxisRange(contIdx, contIdx+1))
            featureName := adult.Data.QuantilesFeatures[contIdx]
            calibrationCtx := ctx.In(fmt.Sprintf("continuous_%d_%s", contIdx, featureName))
            quantiles := adult.Data.Quantiles[contIdx]
            if err := layers.ValidateQuantilesForPWLCalibration(quantiles); err != nil {
                graph.SetError(errors.Wrapf(err, "quantile for features %q invalid", featureName))
                return nil
            }
            calibrated := layers.PieceWiseLinearCalibration(calibrationCtx, split, Const(graph, quantiles), *flagTrainableCalibration)
            if !calibrated.AssertDims(batchSize, 1) { // 2-dim tensor, with batch size as the leading dimension.
                return nil
            }
            allEmbeddings = append(allEmbeddings, calibrated)
        }
    }

    layer := Concatenate(allEmbeddings, -1)
    if !layer.AssertDims(batchSize, -1) { // 2-dim tensor, with batch size as the leading dimension.
        return nil
    }
    
    layer = layers.DenseWithBias(ctx.In(fmt.Sprintf("DenseLayer_%d", 0)), layer, *flagNumNodes)
    for ii := 1; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("DenseLayer_%d", ii))
        // Add layer with residual connection.
        tmp := Sigmoid(layer)
        if *flagDropoutRate > 0 {
            tmp = layers.Dropout(ctx, tmp, Const(graph, shapes.CastAsDType(*flagDropoutRate, ModelDType)))
        }
        tmp = layers.DenseWithBias(ctx, tmp, *flagNumNodes)
        layer = Add(layer, tmp)  // Residual connections
    }
    layer = Sigmoid(layer)
    logits := layers.DenseWithBias(ctx.In("DenseFinal"), layer, 1)
    if !logits.AssertDims(batchSize, 1) { // 2-dim tensor, with batch size as the leading dimension.
        return nil
    }
    return []*Node{logits}
}

%% --platform=Host
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()

// Let's just check that we get the right shape from the model function, wihtout any real data.
graph := manager.NewGraph("test")
ctx := context.NewContext(manager)
ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

inputs := []*Node{
    // Categorical: shaped [batch_size, num_categorical]
    graph.Parameter("categorical", shapes.Make(shapes.Int64, *flagBatchSize, len(adult.Data.VocabulariesFeatures))),
    // Continuous: shaped [batch_size, num_continuos]
    graph.Parameter("continuous", shapes.Make(shapes.Float32, *flagBatchSize, len(adult.Data.QuantilesFeatures))),
    // Weights: shaped [batch_size, 1]
    graph.Parameter("weights", shapes.Make(shapes.Float32, *flagBatchSize, 1)),    
}
logits := ModelGraph(ctx, nil, inputs)
AssertNoError(graph.Error())
AssertNoError(ctx.Error())
fmt.Printf("Logits shape for batch_size=%d: %s\n", *flagBatchSize, logits[0].Shape())

Logits shape for batch_size=128: (Float32)[128 1]


## Training Loop

We can create a training loop with only a `Manager`, a `Context` (for the model varibles) and the `ModelGraph` function.

To make it more interesting we also add the following:

* Accuracy metrics for training and testing.
* Checkpoints -- so trained model can be saved, and reloaded.
* A progress-bar that also shows training metrics.
* We dynamically plot how the loss and accuracy evolve.

First we define the corresponding flags and the `trainModel` function, and run it for very few steps to make sure
it is working.

In [6]:
import (
    "fmt"
    "io"
    "time"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/shapes"
    "github.com/gomlx/gomlx/types/slices"
    "github.com/gomlx/gomlx/types/tensor"
    "github.com/janpfeifer/gonb/gonbui"
)

var (
    flagOptimizer       = flag.String("optimizer", "adam", "Type of optimizer to use: 'sgd' or 'adam'")
    flagLearningRate    = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagCheckpoint     = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagNumPlotPoints = flag.Int("plot_points", 10, "Number points to plot using Chart.JS.")
)

func trainModel() {
    manager := BuildManager().NumThreads(*flagNumThreads).NumReplicas(*flagNumReplicas).Platform(*flagPlatform).MustDone()
    adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
    trainingDS, trainingEvalDS, testEvalDS := BuildDatasets(manager)

    // Context holds the variables and optionally hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(*flagCheckpoint).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Pick a known optimizer.
    optimizerFn, found := optimizers.KnownOptimizers[*flagOptimizer]
    if !found {
        log.Fatalf("Unknown optimizer %q, please use one of %v",
            *flagOptimizer, slices.Keys(optimizers.KnownOptimizers))
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    trainer := train.NewTrainer(manager, ctx, ModelGraph, losses.BinaryCrossentropyLogits,
        optimizerFn(),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics
    AssertNoError(ctx.Error())

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint.
    if checkpoint != nil {
        train.NTimesDuringLoop(loop, *flagNumPlotPoints, "checkpointing", 100, func(_ *train.Loop, _ []tensor.Tensor) error {
            return checkpoint.Save()
        })
    }

    // Attach a margaid plots.
    if *flagNumPlotPoints > 0 {
        margaid.New(1024, 400, testEvalDS).DynamicUpdates().Attach(loop, *flagNumPlotPoints)
    }

    // Run the given number of steps.
    _, err := loop.RunSteps(trainingDS, *flagNumSteps)
    AssertNoError(err)

    // Print a final evaluation on train and test datasets.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainingEvalDS, testEvalDS)
    AssertNoError(err)
    fmt.Println()
}

// Notice command line flags are passed in the %% notebook command. We set --plot_points=0 here to disable plotting
// since this is only a quick test that our train() loop is working.
%% --platform Host --steps=500 --plot_points=0
trainModel()

Training (500 steps):  100% [========================================] (211 steps/s) [loss=0.347] [~loss=0.357] [~acc=84.10%]        

Results on train:
	Mean Loss (#loss): 0.343
	Mean Accuracy (#acc): 84.63%
Results on test:
	Mean Loss (#loss): 0.340
	Mean Accuracy (#acc): 85.04%



## Final run

With everything working, we can do our final run.

> **Note** here is where someone might want to hyperparameter tune, trying out different hyperparameters.

In [7]:
%% --platform Host --steps=5000 --plot_points=50 
trainModel()

Training (5000 steps):  100% [========================================] (537 steps/s) [loss=0.262] [~loss=0.276] [~acc=87.10%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L3.448643e+01,5.200000e+00 L5.146263e+01,1.591192e+02 L6.843882e+01,1.850606e+02 L8.541502e+01,2.079423e+02 L1.023912e+02,2.164564e+02 L1.193674e+02,2.153921e+02 L1.363436e+02,2.141948e+02 L1.533198e+02,2.184518e+02 L1.702960e+02,2.247045e+02 L1.872722e+02,2.212455e+02 L2.042484e+02,2.245714e+02 L2.212246e+02,2.240392e+02 L2.382008e+02,2.241723e+02 L2.551770e+02,2.237732e+02 L2.721532e+02,2.243053e+02 L2.891294e+02,2.251035e+02 L3.061056e+02,2.276311e+02 L3.230818e+02,2.245714e+02 L3.400579e+02,2.176536e+02 L3.570341e+02,2.314891e+02 L3.740103e+02,2.340167e+02 L3.909865e+02,2.329524e+02 L4.079627e+02,2.325534e+02 L4.249389e+02,2.317552e+02 L4.419151e+02,2.314891e+02 L4.588913e+02,2.320212e+02 L4.758675e+02,2.344159e+02 L4.928437e+02,2.358792e+02 L5.098199e+02,2.358792e+02 L5.267961e+02,2.353471e+02 L5.437723e+02,2.346820e+02 L5.607485e+02,2.353471e+02 L5.777247e+02,2.348149e+02 L5.947009e+02,2.346820e+02 L6.116771e+02,2.160573e+02 L6.286533e+02,2.332185e+02 L6.456295e+02,2.286954e+02 L6.626057e+02,2.370765e+02 L6.795819e+02,2.372096e+02 L6.965580e+02,2.348149e+02 L7.135342e+02,2.374756e+02 L7.305104e+02,2.376086e+02 L7.474866e+02,2.350810e+02 L7.644628e+02,2.366774e+02 L7.814390e+02,2.364114e+02 L7.984152e+02,2.358792e+02 L8.153914e+02,2.354802e+02 L8.323676e+02,2.354802e+02 L8.493438e+02,2.362783e+02 L8.663200e+02,2.364114e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.107367e+02 L3.448643e+01,1.580891e+01 L5.146263e+01,7.441387e+01 L6.843882e+01,1.397451e+02 L8.541502e+01,1.727046e+02 L1.023912e+02,1.918217e+02 L1.193674e+02,2.066888e+02 L1.363436e+02,2.136653e+02 L1.533198e+02,2.199039e+02 L1.702960e+02,2.174371e+02 L1.872722e+02,2.150910e+02 L2.042484e+02,2.205223e+02 L2.212246e+02,2.238289e+02 L2.382008e+02,2.263793e+02 L2.551770e+02,2.278734e+02 L2.721532e+02,2.338215e+02 L2.891294e+02,2.336240e+02 L3.061056e+02,2.320202e+02 L3.230818e+02,2.302687e+02 L3.400579e+02,2.284555e+02 L3.570341e+02,2.268758e+02 L3.740103e+02,2.319633e+02 L3.909865e+02,2.374613e+02 L4.079627e+02,2.366471e+02 L4.249389e+02,2.344173e+02 L4.419151e+02,2.449590e+02 L4.588913e+02,2.452554e+02 L4.758675e+02,2.463492e+02 L4.928437e+02,2.438794e+02 L5.098199e+02,2.430051e+02 L5.267961e+02,2.374147e+02 L5.437723e+02,2.479388e+02 L5.607485e+02,2.473562e+02 L5.777247e+02,2.370690e+02 L5.947009e+02,2.394780e+02 L6.116771e+02,2.344328e+02 L6.286533e+02,2.305584e+02 L6.456295e+02,2.429810e+02 L6.626057e+02,2.444053e+02 L6.795819e+02,2.365739e+02 L6.965580e+02,2.468246e+02 L7.135342e+02,2.413631e+02 L7.305104e+02,2.503233e+02 L7.474866e+02,2.470797e+02 L7.644628e+02,2.548000e+02 L7.814390e+02,2.435546e+02 L7.984152e+02,2.445686e+02 L8.153914e+02,2.435584e+02 L8.323676e+02,2.397777e+02 L8.493438e+02,2.452842e+02 L8.663200e+02,2.374525e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M6.860859e+01,0 L6.860859e+01,-6 M1.195372e+02,0 L1.195372e+02,-6 M1.704658e+02,0 L1.704658e+02,-6 M2.213943e+02,0 L2.213943e+02,-6 M2.723229e+02,0 L2.723229e+02,-6 M3.232515e+02,0 L3.232515e+02,-6 M3.741801e+02,0 L3.741801e+02,-6 M4.251087e+02,0 L4.251087e+02,-6 M4.760373e+02,0 L4.760373e+02,-6 M5.269659e+02,0 L5.269659e+02,-6 M5.778944e+02,0 L5.778944e+02,-6 M6.288230e+02,0 L6.288230e+02,-6 M6.797516e+02,0 L6.797516e+02,-6 M7.306802e+02,0 L7.306802e+02,-6 M7.816088e+02,0 L7.816088e+02,-6 M8.325374e+02,0 L8.325374e+02,-6 "/> 0 300 600 900 1200 1500 1800 2100 2400 2700 3000 3300 3600 3900 4200 4500 4800 Steps 0.780 0.810 0.840 0.870 accuracy accuracy metrics Eval on test: Mean Accuracy Train: Moving Average Accuracy

<path d="M1.768000e+01,2.501884e+02 L3.448643e+01,1.807707e+02 L5.146263e+01,9.927364e+01 L6.843882e+01,8.405822e+01 L8.541502e+01,7.668827e+01 L1.023912e+02,7.133197e+01 L1.193674e+02,6.803657e+01 L1.363436e+02,6.626115e+01 L1.533198e+02,6.399021e+01 L1.702960e+02,6.327621e+01 L1.872722e+02,6.270592e+01 L2.042484e+02,6.157862e+01 L2.212246e+02,6.166339e+01 L2.382008e+02,6.101461e+01 L2.551770e+02,6.082591e+01 L2.721532e+02,6.072124e+01 L2.891294e+02,6.068269e+01 L3.061056e+02,6.064413e+01 L3.230818e+02,6.042393e+01 L3.400579e+02,6.055457e+01 L3.570341e+02,5.667721e+01 L3.740103e+02,5.664276e+01 L3.909865e+02,5.550966e+01 L4.079627e+02,5.488881e+01 L4.249389e+02,5.654410e+01 L4.419151e+02,5.449967e+01 L4.588913e+02,5.458385e+01 L4.758675e+02,5.409834e+01 L4.928437e+02,5.385912e+01 L5.098199e+02,5.384551e+01 L5.267961e+02,5.371295e+01 L5.437723e+02,5.375616e+01 L5.607485e+02,5.376628e+01 L5.777247e+02,5.373838e+01 L5.947009e+02,5.329420e+01 L6.116771e+02,5.917749e+01 L6.286533e+02,5.354700e+01 L6.456295e+02,5.423709e+01 L6.626057e+02,5.269674e+01 L6.795819e+02,5.205680e+01 L6.965580e+02,5.418895e+01 L7.135342e+02,5.192150e+01 L7.305104e+02,5.184105e+01 L7.474866e+02,5.154460e+01 L7.644628e+02,5.169269e+01 L7.814390e+02,5.138132e+01 L7.984152e+02,5.138470e+01 L8.153914e+02,5.136074e+01 L8.323676e+02,5.134651e+01 L8.493438e+02,5.131255e+01 L8.663200e+02,5.142442e+01 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.448643e+01,1.815754e+02 L5.146263e+01,1.067606e+02 L6.843882e+01,7.282980e+01 L8.541502e+01,8.251753e+01 L1.023912e+02,4.022748e+01 L1.193674e+02,4.359631e+01 L1.363436e+02,6.885063e+01 L1.533198e+02,5.316687e+01 L1.702960e+02,9.196719e+01 L1.872722e+02,6.212496e+01 L2.042484e+02,6.919079e+01 L2.212246e+02,3.581107e+01 L2.382008e+02,3.419248e+01 L2.551770e+02,3.769127e+01 L2.721532e+02,6.239877e+01 L2.891294e+02,7.231696e+01 L3.061056e+02,6.251503e+01 L3.230818e+02,6.463281e+01 L3.400579e+02,8.936667e+01 L3.570341e+02,5.099956e+01 L3.740103e+02,1.365593e+01 L3.909865e+02,3.121255e+01 L4.079627e+02,1.146751e+02 L4.249389e+02,4.323158e+01 L4.419151e+02,5.142997e+01 L4.588913e+02,4.533687e+01 L4.758675e+02,2.697736e+01 L4.928437e+02,5.451394e+01 L5.098199e+02,8.856639e+01 L5.267961e+02,3.702534e+01 L5.437723e+02,5.200000e+00 L5.607485e+02,4.783554e+01 L5.777247e+02,1.050420e+02 L5.947009e+02,4.134955e+01 L6.116771e+02,8.906940e+01 L6.286533e+02,5.069471e+01 L6.456295e+02,4.199397e+01 L6.626057e+02,7.397339e+01 L6.795819e+02,8.299074e+01 L6.965580e+02,6.569183e+01 L7.135342e+02,7.047905e+01 L7.305104e+02,3.680330e+01 L7.474866e+02,1.929762e+01 L7.644628e+02,2.574347e+01 L7.814390e+02,6.030941e+01 L7.984152e+02,4.272873e+01 L8.153914e+02,5.739784e+01 L8.323676e+02,5.602358e+01 L8.493438e+02,4.285074e+01 L8.663200e+02,4.196539e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L3.448643e+01,1.950716e+02 L5.146263e+01,1.493578e+02 L6.843882e+01,1.117395e+02 L8.541502e+01,9.318898e+01 L1.023912e+02,8.181414e+01 L1.193674e+02,7.347909e+01 L1.363436e+02,6.992749e+01 L1.533198e+02,6.558044e+01 L1.702960e+02,6.493105e+01 L1.872722e+02,6.626196e+01 L2.042484e+02,6.239026e+01 L2.212246e+02,6.038839e+01 L2.382008e+02,5.845678e+01 L2.551770e+02,5.761904e+01 L2.721532e+02,5.749869e+01 L2.891294e+02,5.753261e+01 L3.061056e+02,5.820273e+01 L3.230818e+02,5.839085e+01 L3.400579e+02,5.846592e+01 L3.570341e+02,5.745228e+01 L3.740103e+02,5.519222e+01 L3.909865e+02,5.225927e+01 L4.079627e+02,5.225991e+01 L4.249389e+02,5.348646e+01 L4.419151e+02,4.872026e+01 L4.588913e+02,4.805669e+01 L4.758675e+02,4.854029e+01 L4.928437e+02,4.906702e+01 L5.098199e+02,5.139550e+01 L5.267961e+02,5.357887e+01 L5.437723e+02,4.786053e+01 L5.607485e+02,4.773158e+01 L5.777247e+02,5.153753e+01 L5.947009e+02,5.062051e+01 L6.116771e+02,5.213498e+01 L6.286533e+02,5.361520e+01 L6.456295e+02,5.048427e+01 L6.626057e+02,4.938586e+01 L6.795819e+02,5.111704e+01 L6.965580e+02,4.730869e+01


Results on train:
	Mean Loss (#loss): 0.274
	Mean Accuracy (#acc): 87.33%
Results on test:
	Mean Loss (#loss): 0.280
	Mean Accuracy (#acc): 87.05%

